In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from sklearn.model_selection import train_test_split
import yaml
import torch
import cv2

In [ ]:
# Path to the images and annotations directories
images_dir = 'images'
annotations_dir = 'annotations'

# List a few files from each directory
image_files = os.listdir(images_dir)
annotation_files = os.listdir(annotations_dir)

In [ ]:
# Function to parse an XML file and extract information
def parse_annotation(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    annotation_data = {}
    
    # Extracting relevant information from the XML
    for elem in root:
        if elem.tag == 'filename':
            annotation_data['filename'] = elem.text
        elif elem.tag == 'size':
            annotation_data['size'] = {child.tag: int(child.text) for child in elem}
        elif elem.tag == 'object':
            obj_data = {}
            for child in elem:
                if child.tag == 'name':
                    obj_data['class'] = child.text
                elif child.tag == 'bndbox':
                    obj_data['bndbox'] = {grandchild.tag: int(grandchild.text) for grandchild in child}
            if 'objects' not in annotation_data:
                annotation_data['objects'] = []
            annotation_data['objects'].append(obj_data)

    return annotation_data

In [ ]:
def yolo_format_bbox(img_width, img_height, bbox):
    """
    Convert bounding box to YOLO format.
    """
    xmin, ymin, xmax, ymax = bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return [x_center, y_center, width, height]

In [ ]:
# Function to resize an image and its corresponding bounding boxes
def resize_image_and_bboxes(image_path, annotation, target_size=(224, 224)):
    # Open the image
    with Image.open(image_path) as img:
        original_size = img.size
        # Resize the image
        img_resized = img.resize(target_size)

    # Calculate resize ratio
    ratio = (target_size[0] / original_size[0], target_size[1] / original_size[1])
    # Update bounding box coordinates
    yolo_annotations = []

    for obj in annotation['objects']:
        xmin = int(obj['bndbox']['xmin'] * ratio[0])
        ymin = int(obj['bndbox']['ymin'] * ratio[1])
        xmax = int(obj['bndbox']['xmax'] * ratio[0])
        ymax = int(obj['bndbox']['ymax'] * ratio[1])
        
        # Convert bbox to YOLO format
        yolo_bbox = yolo_format_bbox(target_size[0], target_size[1], {'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax})
        yolo_annotations.append({'class': obj['class'], 'bndbox': yolo_bbox})

    return img_resized, yolo_annotations

In [ ]:
class_to_id = {
    'crosswalk': 0,
    'speedlimit': 1,
    'stop': 2,
    'trafficlight': 3
}

In [ ]:
def process_dataset(images_dir, annotations_dir, output_dir, target_size=(224, 224)):
    image_files = os.listdir(images_dir)
    annotation_files = os.listdir(annotations_dir)

    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

    # Split the dataset
    train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

    for subset, files in [('train', train_files), ('val', val_files), ('test', test_files)]:
        for file in files:
            image_path = os.path.join(images_dir, file)
            annotation_path = os.path.join(annotations_dir, file.replace('.png', '.xml'))

            annotation = parse_annotation(annotation_path)
            resized_image, resized_annotations = resize_image_and_bboxes(image_path, annotation, target_size)

            # Save the resized image
            resized_image.save(os.path.join(output_dir, subset, file))

            # Save the resized annotations in YOLO format
            with open(os.path.join(output_dir, subset, file.replace('.png', '.txt')), 'w') as f:
                for obj in resized_annotations:
                    bbox = obj['bndbox']
                    class_id = class_to_id[obj['class']]  # You need to map class names to IDs
                    f.write(f"{class_id} {' '.join(map(str, bbox))}\n")


In [ ]:
output_dir = 'output/'

In [ ]:
process_dataset(images_dir, annotations_dir, output_dir)

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%pip install -r yolov5/requirements.txt

In [ ]:
import yaml

data_yaml = {
    'train': 'output/train',
    'val': 'output/val',
    'test': 'output/test',
    'nc': 4,
    'names': ['crosswalk', 'speedlimit', 'stop', 'trafficlight']
}

yaml_file = 'output/dataset.yaml'  # Update this path

with open(yaml_file, 'w') as file:
    yaml.dump(data_yaml, file, sort_keys=False)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
!python yolov5/train.py --img 320 --batch 16 --epochs 50 --data output/dataset.yaml --weights yolov5s.pt


In [ ]:
!python yolov5/val.py --weights yolov5/runs/train/exp/weights/best.pt --data output/dataset.yaml --img 320


In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp/weights/best.pt --img 320 --conf 0.4 --source output/test


In [ ]:
%load_ext tensorboard
%tensorboard --logdir output/runs